# **Single-call**

When you have a small number of compounds, a single-call for each compound can work effectively

import requests

In [ ]:
cids_list = [10339994, 10807095, 5330258] 

def SmilesFromCID(cids_list):
    results = {'cids': [], 'smiles': []}

    base_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/CanonicalSMILES/JSON'
    for cid in cids_list:
        url = base_url.format(cid)
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            smiles = data["PropertyTable"]["Properties"][0]["CanonicalSMILES"]
            results['cids'].append(cid)
            results['smiles'].append(smiles)
        else:
            print(f"Failed to retrieve SMILES for CID {cid}")
        
    return results

# **Batch-call**
When you have a large dataset, around hundred thousand compounds, a single-call are repeated many times and pontentially reach the API call limit of PubChem. Therefore, the batch-call is the more efficient approach in this case. 

In [ ]:
# cids_list contains hundreds thousand compounds

def SmilesFromCID_batch(cids_list, batch_size=50):
    results = {'smiles': [], 'cids': []}

    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/property/CanonicalSMILES/JSON"
    for i in range(0, len(cids), batch_size):
        batch = cids[i:i+batch_size]
        try:
            response = requests.post(base_url, data={"cid": ",".join(map(str, batch))}, timeout=10)

            if response.status_code == 200:
                data = response.json()
                for prop in data["PropertyTable"]["Properties"]:
                    smiles_list['smiles'].append(prop["CanonicalSMILES"])
                    smiles_list['cids'].append(prop["CID"])
                print(f"Batch {i//batch_size + 1}: Success ({len(batch)} compounds)") 
            else:
                print(f"Batch {i//batch_size + 1}: Failed with status {response.status_code}")

        except requests.exceptions.RequestException as e:
            print(f"Batch {i//batch_size + 1}: Request failed - {e}")
            break 

        time.sleep(1.5)  # Small delay between batches to prevent rate limits

    return smiles_list

